# CNN for Detection of Overlapping Cells

### Import Packages

In [ ]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

from tqdm import tqdm

### Create Training Data

In [ ]:
def createImg(num_of_objects):
    
    global im, sizes, intensities
    
    im = np.random.randint(165, 200, size = (100, 100)).astype('uint8')
    obj_num = num_of_objects
    circle_centers = np.random.randint(25, 75, size = (obj_num, 2))
    intensities = np.random.randint(80, 90, size = (obj_num, 1)).astype('int')
    sizes = np.random.randint(8, 12, size = (obj_num, 1)).astype('int')

    for jj in range(len(circle_centers)):
        cv2.circle(im, (circle_centers[jj][0], circle_centers[jj][1]), sizes[jj], int(intensities[jj]), -1)
        
    ## blur image
    im = cv2.medianBlur(im, 3)
    im = cv2.GaussianBlur(im, (3,3), 9)
        
    
    return im

In [ ]:
training_data = []
imgSize = 100

def create_training_data():
    for j in range(0, 30000):
        num_obj = np.random.randint(1,4)
        img = createImg(num_obj)
        label = int(num_obj-1)
        ret, img2 = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        training_data.append([img2, label])
        
        
create_training_data()
len(training_data)

import random
random.shuffle(training_data)

### Format Data for Neural Network

In [ ]:
X = []
y = []

for features, labels in training_data:
    X.append(features)
    y.append(labels)
  
  
X = np.array(X).reshape(-1, imgSize, imgSize, 1)
y = np.array(y)

print(X.shape)
print(y.shape)

from sklearn.preprocessing import MultiLabelBinarizer, LabelBinarizer
mlb = LabelBinarizer()
y = mlb.fit_transform(y)

### Create Training and Validation Sets

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, UpSampling2D
from keras.callbacks import LearningRateScheduler
from keras import optimizers

def step_decay(epoch):
    initial_lrate = 0.1
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

lrate = LearningRateScheduler(step_decay)
callbacks_list = [lrate]


adam = optimizers.Adam(lr=0.001)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = tf.keras.utils.normalize(X_train, axis=1)
X_test = tf.keras.utils.normalize(X_test, axis=1)

In [ ]:
# delete original arrays to lower RAM 
del X, y

### Define Neural Network Model and Train Network!

In [ ]:
model = Sequential()
model.add(Conv2D(128, (3,3), input_shape = X_train.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(16, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (1,1)))
model.add(Activation("relu"))
model.add(UpSampling2D(size=(2,2)))

model.add(Conv2D(64, (1,1)))
model.add(Activation("relu"))
model.add(UpSampling2D(size=(2,2)))

model.add(Flatten())
#model.add(Dense(32))
model.add(Dense(32))

model.add(Dense(3))
model.add(Activation("softmax"))



model.compile(loss="mse", 
              optimizer=adam, 
              metrics=["accuracy"])

model.fit(X_train, y_train, batch_size=32, epochs=15)

### Evaluate Network Performance

In [ ]:
model.evaluate(X_test, y_test)

### Save Network Model

In [ ]:
model.save('model_save.h5')
print("Saved model to disk")

# This saved model can now be imported into SimpylCellCounter 